In [1]:
from neo4j import GraphDatabase
from pandas import DataFrame
import os

driver = GraphDatabase.driver("bolt://localhost:7687", auth=("neo4j", "1004"), encrypted=False)
session = driver.session()
print("Ricardo Jara")
print("Lista de provincias")
result = session.run("""MATCH (n) return n.name""")
DataFrame(result)


Ricardo Jara
Lista de provincias


,0
0,Loja
1,GUayaquil
2,Colegios
3,Iglesias
4,Restaurants
...,...
272,Parque Japones
273,None
274,None
275,None


In [15]:

def A(origen, destino):
    return session.run("""
    MATCH (start:Station {name: '"""+origen+"""'}), (end:Station {name: '"""+destino+"""'})
    CALL gds.alpha.shortestPath.astar.stream({
 nodeQuery: 'MATCH (p:Station) RETURN id(p) AS id',
 relationshipQuery: 'MATCH (p1:Station)-[r:CONNECTION]->(p2:Station) RETURN id(p1) AS source, id(p2) AS target, r.time AS weight',
 startNode: start,
 endNode: end,
 relationshipWeightProperty: 'weight',
 propertyKeyLat: 'latitude',
 propertyKeyLat: 'longitude'
})YIELD nodeId, cost
RETURN gds.util.asNode(nodeId).name AS Provincia, cost as Distancia
""")

def costo(origen, destino):
    return session.run("""
    MATCH (start:Station {name: '"""+origen+"""'}), (end:Station {name: '"""+destino+"""'})
    CALL gds.alpha.shortestPath.stream({
      nodeProjection: 'Station',
      relationshipProjection: {
        ROAD: {
          type: 'CONNECTION',
          properties: 'time',
          orientation: 'UNDIRECTED'
        }
      },
      startNode: start,
      endNode: end,
      relationshipWeightProperty: 'time'
    })
    YIELD nodeId, cost
    RETURN gds.util.asNode(nodeId).name AS Provincia, cost AS costo
""")

def amplitud(origen, destino):
    return session.run("""
    MATCH (a:Station{name:'"""+origen+"""'})
    WITH id(a) AS startNode
    CALL gds.alpha.bfs.stream('Pais', {startNode: startNode, maxDepth: """+destino+"""})
    YIELD path
    UNWIND [ n in nodes(path) | n.name] AS tags
    RETURN tags
    ORDER BY tags
""")

def profundiad(origen, destino):
    return session.run("""
    MATCH (a:Station{name:'"""+origen+"""'}), (d:Station{name:'"""+destino+"""'})
    WITH id(a) AS startNode, [id(d)] AS targetNodes
    CALL gds.alpha.dfs.stream('Pais', {startNode: startNode, targetNodes: targetNodes})
    YIELD path
    UNWIND [ n in nodes(path) | n.name ] AS tags
    RETURN tags as Provincia
    ORDER BY tags DESC
""")


origen = ""
destino= ""

print ("Selecciona una opción")
print ("\t1 - •A* ")
print ("\t2 - •Costo")
print ("\t3 - •Amplitud")
print ("\t4 - •Profundidad ")
print ("\t9 - salir")
opcionMenu = input(" Ingrese opcion ")
if opcionMenu=="1":
    origen =  input(" Ingrese Origen ")
    destino=  input(" Ingrese Destino ")
    result = A(origen,destino)
elif opcionMenu=="2":
    origen =  input(" Ingrese Origen ")
    destino=  input(" Destino")
    result = costo(origen,destino)
elif opcionMenu=="3":
    origen =  input(" Ingrese Origen ")
    destino=  input(" Ingrese Numero de Niveles")
    result = amplitud(origen,destino)
elif opcionMenu=="4":
    origen =  input(" Ingrese Origen ")
    destino=  input(" Ingrese Destino ")
    result = profundiad(origen,destino)
elif opcionMenu=="9":
    print ("")
    print("Saliendo")

DataFrame(result)

Selecciona una opción
	1 - •A* 
	2 - •Costo
	3 - •Amplitud
	4 - •Profundidad 
	9 - salir
 Ingrese opcion 4
 Ingrese Origen Ecuador
 Ingrese Destino Manta


,0
0,nacional primero de abril
1,Upc San Sebastian
2,Upc Barreiro
3,Upc
4,Upc
...,...
178,Balcon de Imbabura
179,Babahoyo
180,Babahoyo
181,Alessandro Volta
